In [39]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [40]:

import sys

import ipywidgets as widgets

sys.path.append('../')
sys.path.append('../src')

import src.backend
import src.workload_builder as builder
from mbi import Domain
import src.plots
import altair as alt
alt.renderers.enable('default')
from IPython.display import display, clear_output

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


alt.data_transformers.disable_max_rows()

/tmp/ipykernel_70381/218976982.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


DataTransformerRegistry.enable('default')

In [41]:
data_path = '../data/CPS/CPS.csv'
cps_domain = Domain(attrs=('age','income','marital','race'), shape=(100,100,7,4))
questions = ["Warm Up Question: Type 'Yes' if you are ready to begin!",
            "Question 1/6: How many Chinese and American Indians have 'Widowed' as their marital status?",
            "Question 2/6: How many people over the age of 65 have income between 100,000 and 149,000?",
            "Question 3/6: What is the second most common marital status of people with income in the 100,000 to 199,000 range?",
            "Question 4/6: Choose the top 3 age groups that have marital status: 'Never Married'?",
            "Question 5/6: In the 100,000 to 199,000 income range, rank the top 3 races in terms of count from greatest to least.",
            "Question 6/6: What is the second most common race in the 45-54 age group?"]

marital_race = {'marital':1, 'race':1}
age_income = {'age':10, 'income':10}
marital_income = {'income':10, 'marital':1}
age_marital = {'age':10, 'marital':1}
race_income = {'income':10, 'race':1}
age_race = {'age':10, 'race':1}

visualizations = [None, marital_race, age_income, marital_income, age_marital, race_income, age_race]
prev_spec = None
curr_spec = None
epsilon_increments = [None,0.05,0.05,0.05,0.05,0.05,0.05,0.05]
budget = 1.0
max_tries = 10

index = 0
tries = 0
back_end = None


In [42]:
cps_domain

Domain(age: 100, income: 100, marital: 7, race: 4)

# Logging

In [43]:
import logging

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%',
            'height': '160px',
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout',
            'output_type': 'stream',
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs

    def show_logs(self):
        """ Show the logs """
        display(self.out)

    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()


logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [44]:
plot_output1 = widgets.Output()
plot_output2 = widgets.Output()
plot_output3 = widgets.Output()
plot_output4 = widgets.Output()
plot_output5 = widgets.Output()
plot_output6 = widgets.Output()

In [45]:
plot_output_rm1 = widgets.Output()
plot_output_rm2 = widgets.Output()
plot_output_rm3 = widgets.Output()
plot_output_rm4 = widgets.Output()
plot_output_rm5 = widgets.Output()
plot_output_rm6 = widgets.Output()

In [46]:
plot_output_after = widgets.Output()

In [47]:
plot_output = widgets.Output()

In [48]:
# plot_output

 ## Dropdown Boxes

In [49]:
columns = ['Age','Income','Marital Status']
translate = {'Income': 'income', 'Marital Status': 'marital', 'Age': 'age', 'Race':'race'}

Text Boxes

In [50]:
import IPython
from ipywidgets import Layout

global index

IPython.display.HTML('''<style>
.mytext > select,.mytext > textarea   {
    font-style: italic;
    color: blue;
    font-size: 30px;
}
</style>''')
# w2=widgets.Textarea(value="Here it is")
# w2.add_class("mytext")

# prompt = widgets.Label(
#     value= questions[0],
#     placeholder='Type',
#     font_weight='bold',
#     # description=questions[0],
#     # style = {'button_color': 'transparent', 'font_size': "20px"},
#     disabled=True,
#     layout=widgets.Layout(width='1000px', height='30px', font_size='200px')
# )

prompt = widgets.HTML(
    value="<b>" + questions[0] + "</b>",
    placeholder='',
    description='',
)

prompt2 = widgets.HTML(
    value="<b>" + questions[1] + "</b>",
    placeholder='',
    description='',
)

prompt3 = widgets.HTML(
    value="<b>" + questions[2] + "</b>",
    placeholder='',
    description='',
)

prompt4 = widgets.HTML(
    value="<b>" + questions[3] + "</b>",
    placeholder='',
    description='',
)

prompt5 = widgets.HTML(
    value="<b>" + questions[4] + "</b>",
    placeholder='',
    description='',
)

prompt6 = widgets.HTML(
    value="<b>" + questions[5] + "</b>",
    placeholder='',
    description='',
)

prompt7 = widgets.HTML(
    value="<b>" + questions[6] + "</b>",
    placeholder='',
    description='',
)
# prompt = widgets.Text(description="", value=questions[0], disabled=True, layout=Layout(width='40%', height='30px'), style=dict(
#     font_style='italic',
#     font_size='20px',
#     font_weight='bold',
#     font_variant="small-caps",
#     text_color='red',
#     text_decoration='underline'
# ))

# prompt2 = widgets.Button(description=questions[1], layout=Layout(width='55%', height='30px'), style=dict(
#     font_style='italic',
#     font_size='20px',
#     font_weight='bold',
#     font_variant="small-caps",
#     text_color='red',
#     text_decoration='underline'
# ))
#
# prompt3 = widgets.Button(description=questions[2], layout=Layout(width='55%', height='30px'), style=dict(
#     font_style='italic',
#     font_size='20px',
#     font_weight='bold',
#     font_variant="small-caps",
#     text_color='red',
#     text_decoration='underline'
# ))
#
# prompt4 = widgets.Button(description=questions[3], layout=Layout(width='55%', height='30px'), style=dict(
#     font_style='italic',
#     font_size='20px',
#     font_weight='bold',
#     font_variant="small-caps",
#     text_color='red',
#     text_decoration='underline'
# ))
#
# prompt5 = widgets.Button(description=questions[4], layout=Layout(width='55%', height='30px'), style=dict(
#     font_style='italic',
#     font_size='20px',
#     font_weight='bold',
#     font_variant="small-caps",
#     text_color='red',
#     text_decoration='underline'
# ))
#
# prompt6 = widgets.Button(description=questions[5], layout=Layout(width='55%', height='30px'), style=dict(
#     font_style='italic',
#     font_size='20px',
#     font_weight='bold',
#     font_variant="small-caps",
#     text_color='red',
#     text_decoration='underline'
# ))
#
# prompt7 = widgets.Button(description=questions[6], layout=Layout(width='55%', height='30px'), style=dict(
#     font_style='italic',
#     font_size='20px',
#     font_weight='bold',
#     font_variant="small-caps",
#     text_color='red',
#     text_decoration='underline'
# ))

# finished_tries = widgets.Textarea(
#     value="",
#     placeholder='Type',
#     description='',
#     disabled=True,
#     layout=widgets.Layout(width='500px', height='30px') #500,30
# )

answer = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='300px', height='40px')
)

answer2 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='500px', height='50px')
)

answer3 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='500px', height='50px')
)

answer4 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='500px', height='50px')
)

answer5 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='500px', height='50px')
)

answer6 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='500px', height='50px')
)

answer7 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='500px', height='50px')
)

In [51]:
def make_plots(idd):
    for i in range(len(questions) - 1):
        if i == idd:
            continue
        # with plot_output_arr[i]:
        column_names = list(visualizations[i+1].keys())
        plot = src.plots.linked_hist(column_names[0], column_names[1], data=current_spec_arr[i].reset_index(column_names), display_true=False, history=False)
        logger.info(plot)
        # print("hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh")

        bar_label = widgets.HBox([budget_spent, progress_bar, remeasure_arr[i]])
        ques = widgets.HBox([prompt_arr[i+1]])
        # prompt_answer = widgets.VBox([answer_arr[i+1], submit_btn], layout=box_layout)
        prompt_answer = widgets.VBox([answer_arr[i+1]], layout=box_layout)
        display(bar_label)
        display(ques)

        display(plot)
        histogram_plots.append(widgets.VBox([plot_output_arr[i]]))
        display(histogram_plots[i])
        interface = widgets.VBox([prompt_answer])
        display(interface)


In [52]:
plot_output_arr = [plot_output1, plot_output2, plot_output3, plot_output4, plot_output5, plot_output6]

## Buttons

In [53]:
import copy
import numpy as np
from ipywidgets import Layout

submit_btn = widgets.Button(description='Start')
submit_btn.style.button_color = '#C6C6C6'
# submit_btn_final = widgets.Button(description='Submit', layout=Layout(width='20%', height='50px'))
submit_btn_final = widgets.Button(description='Submit All')
submit_btn_final.style.button_color = '#C6C6C6'
# make_it_better1 = widgets.Button(description='Remeasure Q1', button_style='warning')
make_it_better1 = widgets.Button(description='Remeasure Q1')
make_it_better2 = widgets.Button(description='Remeasure Q2')
make_it_better3 = widgets.Button(description='Remeasure Q3')
make_it_better4 = widgets.Button(description='Remeasure Q4')
make_it_better5 = widgets.Button(description='Remeasure Q5')
make_it_better6 = widgets.Button(description='Remeasure Q6')
rm_button_arr = [make_it_better1, make_it_better2, make_it_better3, make_it_better4, make_it_better5, make_it_better6]

separator = widgets.Button(description='', layout=Layout(width='99%', height='1px'))
separator.style.button_color = 'black'
for i in range(len(questions)-1):
    rm_button_arr[i].style.button_color = '#C6C6C6'

gap_button = widgets.Button(description='', layout=Layout(width='95%', height='50px'), disabled=True)
gap_button.style.button_color = 'white'
gap_button_Q = widgets.Button(description='', layout=Layout(width='95%', height='1px'), disabled=True)
gap_button_Q.style.button_color = 'white'

align_button = widgets.Button(description='', layout=Layout(width='40%', height='30px'), disabled=True)
align_button.style.button_color = 'white'
align_button_remeasure = widgets.Button(description='', layout=Layout(width='4%', height='1px'), disabled=True)
align_button_remeasure.style.button_color = 'white'

prompt_arr = [prompt, prompt2, prompt3, prompt4, prompt5, prompt6, prompt7]
answer_arr = [answer, answer2, answer3, answer4, answer5, answer6, answer7]

remeasure_arr = [make_it_better1, make_it_better2, make_it_better3, make_it_better4, make_it_better5, make_it_better6]

plot_output_rm_arr = [plot_output_rm1, plot_output_rm2, plot_output_rm3, plot_output_rm4, plot_output_rm5, plot_output_rm6]

progress_bar = widgets.FloatProgress(min=0.0, max=max_tries, layout=Layout(width='30%', height='30px'), bar_style='warning') # instantiate the bar
# progress_bar.style.bar_color = '#8B4000'
progress_bar.description = str(tries) + '/' + str(max_tries)
# budget_spent = widgets.Label(value='Number of Remeasures Used So Far', style=dict(font_weight='bold'))

budget_spent = widgets.HTML(
    value="<b>Number of Remeasures are Used So Far</b>",
    placeholder='',
    description='',
)

num_answers = len(questions)
survey_answers = [0]*num_answers

current_spec_arr = []
current_spec_remeasure = []
hist_workloads = []
histogram_plots = []
# hist_plot = None


def make_vis(do_not_update=None):
    global index, survey_answers, visualizations, epsilon_increments, curr_spec, back_end, bar_label, interface, ques
    back_end = src.backend.initialize_backend(cps_domain, data_path, budget=1.0)

    seed = 123
    for i in range(len(questions) - 1):
        # prompt_arr[i+1].value = questions[i+1]
        hist = builder.histogram_workload(cps_domain.config, bin_widths=visualizations[i+1])
        hist_workloads.append(hist)

        y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon_increments[i+1], restarts=20, seed=seed) # got y_hat

        column_names = list(visualizations[i+1].keys())
        curr_spec, cached = back_end.display(hist)
        current_spec_arr.append(curr_spec)
        current_spec_remeasure.append(curr_spec)

        with plot_output_arr[i]:
            plot_output_arr[i].clear_output()
            plot = src.plots.linked_hist(column_names[0], column_names[1], data=curr_spec.reset_index(column_names), display_true=False, history=False)
            # plot_output_arr[i].append_display_data(plot)
            # display(plot_output_arr[i])
        # plot = src.plots.linked_hist(column_names[0], column_names[1], data=curr_spec.reset_index(column_names), display_true=False, history=False)
        # plot_output_arr[i].append_display_data(plot)
        # display(plot_output_arr[i])

        bar_label = widgets.HBox([budget_spent, progress_bar, align_button_remeasure, remeasure_arr[i]])
        ques = widgets.HBox([prompt_arr[i+1]])
        # prompt_answer = widgets.VBox([answer_arr[i+1], submit_btn], layout=box_layout)
        prompt_answer = widgets.VBox([answer_arr[i+1]], layout=box_layout)

        display(separator)
        display(ques)
        display(gap_button_Q)
        display(bar_label)
        display(plot)

        # histogram_plots.append(widgets.VBox([plot_output_arr[i]]))
        # display(histogram_plots[i])
        hist_plot = widgets.VBox([plot_output_arr[i]])
        display(hist_plot)
        interface = widgets.VBox([prompt_answer])
        display(interface)


def on_click_submit(obj):
    global index, survey_answers, visualizations, epsilon_increments, curr_spec, back_end, bar_label, interface, ques, plot
    logger.info('Clicked submit')
    if answer.value == '':
        return
    val = answer.value
    answer.value = answer.placeholder
    survey_answers[index] = val

    interface.close()

    display(gap_button)

    make_vis()

    display(separator)

    display(widgets.HBox([align_button, submit_btn_final]))

    # for i in range(len(questions) - 1):
    #     prompt_arr[i+1].value = questions[i+1]
    #     back_end = src.backend.initialize_backend(cps_domain, data_path, budget=1.0)
    #     hist = builder.histogram_workload(cps_domain.config, bin_widths=visualizations[i+1])
    #     y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon_increments[i+1], restarts=20) # got y_hat
    #     column_names = list(visualizations[i+1].keys())
    #     curr_spec = back_end.display(hist)
    #
    #     with plot_output:
    #         # plot_output.clear_output()
    #         plot = src.plots.linked_hist(column_names[0], column_names[1], data=curr_spec.reset_index(column_names), display_true=False, history=False)
    #         logger.info(plot)
    #
    #     bar_label = widgets.HBox([budget_spent, progress_bar, make_it_better])
    #     ques = widgets.HBox([prompt_arr[i+1]])
    #     prompt_answer = widgets.VBox([answer_arr[i+1], submit_btn], layout=box_layout)
    #     display(bar_label)
    #     display(ques)
    #     display(plot)
    #     interface = widgets.VBox([plot_output, prompt_answer])
    #     display(interface)

    # with plot_output:
    #     clear_output()
    # submit_btn.close()
    # interface.close()
    # bar_label.close()
    # prompt.value = 'Tutorial finished, thanks for your participation!'
    # display(prompt)
    # return

idx = 1

def remeasure_clicked(question_no):
    global index, visualizations, tries, idx, interface, submit_btn,  histogram_plots, plot_output_rm_arr
    logger.info('Clicked remeasure, question: {}'.format(index))
    # clear_output()
    # plot_output_arr[question_no].clear_output()

    key_val = [x for x in visualizations[question_no].items()]


    measure_dict = {'left': key_val[0], 'right': key_val[1]}
    binning(measure_dict, question_no-1, epsilon = epsilon_increments[question_no])

    # for i in range(len(questions) - 1):
    #     key_val = [x for x in visualizations[i+1].items()]
    #     # logger.info(key_val)
    #     measure_dict = {'left': key_val[0], 'right': key_val[1]}
    #     binning(measure_dict, i, epsilon = epsilon_increments[idx])
    # idx += 1

    # make_vis()
    tries += 1
    progress_bar.value = tries
    progress_bar.description = str(tries) + '/' + str(max_tries)

def on_click_make_it_better(obj):
    global index, visualizations, tries, idx, interface, submit_btn, plot_output
    logger.info('Clicked remeasure, question: {}'.format(index))

    with plot_output:
        clear_output()
    # plot_output_before[0].clear_output()
    # print(plot_output_before)
    # submit_btn.close()
    # interface.close()
    # bar_label.close()
    
    key_val = [x for x in visualizations[1].items()]
    measure_dict = {'left': key_val[0], 'right': key_val[1]}
    binning(measure_dict, 0, epsilon = epsilon_increments[1])

    # for i in range(len(questions) - 1):
    #     key_val = [x for x in visualizations[i+1].items()]
    #     # logger.info(key_val)
    #     measure_dict = {'left': key_val[0], 'right': key_val[1]}
    #     binning(measure_dict, i, epsilon = epsilon_increments[idx])
    # idx += 1

    # make_vis()
    tries += 1
    progress_bar.value = tries
    progress_bar.description = str(tries) + '/' + str(max_tries)

def on_click_make_it_better_1(obj):
    remeasure_clicked(1)

def on_click_make_it_better_2(obj):
    remeasure_clicked(2)

def on_click_make_it_better_3(obj):
    remeasure_clicked(3)

def on_click_make_it_better_4(obj):
    remeasure_clicked(4)

def on_click_make_it_better_5(obj):
    remeasure_clicked(5)

def on_click_make_it_better_6(obj):
    remeasure_clicked(6)

submit_btn.on_click(on_click_submit)
make_it_better1.on_click(on_click_make_it_better_1)
make_it_better2.on_click(on_click_make_it_better_2)
make_it_better3.on_click(on_click_make_it_better_3)
make_it_better4.on_click(on_click_make_it_better_4)
make_it_better5.on_click(on_click_make_it_better_5)
make_it_better6.on_click(on_click_make_it_better_6)

box_layout = widgets.Layout(display='flex',
    flex_flow='column',
    align_items='flex-start',
    color='black',
    width='80%')

# bar_label = widgets.HBox([budget_spent, progress_bar, make_it_better])
prompt_answer = widgets.VBox([answer, submit_btn], layout=box_layout)

In [54]:
# import numpy as np
#
# submit_btn = widgets.Button(description='Submit')
# make_it_better = widgets.Button(description='Remeasure')
#
# def on_click_submit(obj):
#     global index, survey_answers, visualizations, epsilon_increments, curr_spec, back_end
#     logger.info('Clicked submit')
#     if answer.value == '':
#         return
#     val = answer.value
#     answer.value = answer.placeholder
#     survey_answers[index] = val
#
#     if index+1 < len(questions):
#         index += 1
#         prompt.value = questions[index]
#     else:
#         with plot_output:
#             clear_output()
#         submit_btn.close()
#         interface.close()
#         bar_label.close()
#         prompt.value = 'Tutorial finished, thanks for your participation!'
#         display(prompt)
#         return
#
#     back_end = src.backend.initialize_backend(cps_domain, data_path, budget=1.0)
#
#     hist = builder.histogram_workload(cps_domain.config, bin_widths=visualizations[index])
#
#     # logger.info(hist.matrix.matrix)
#
#     y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon_increments[index], restarts=20) # got y_hat
#
#     # B = np.linalg.pinv(strategy_matrix)
#
#     # logger.info("printing y_hat")
#     # logger.info(B)
#     logger.info("printing strategy matrix")
# #     logger.info(strategy_matrix.matrix)
#
#     column_names = list(visualizations[index].keys())
#     curr_spec = back_end.display(hist)
#
#     with plot_output:
#         plot_output.clear_output()
#         plot = src.plots.linked_hist(column_names[0], column_names[1], data=curr_spec.reset_index(column_names), display_true=False, history=False)
#         logger.info(plot)
# #         display("Updating...")
#         display(plot)
# #         altair_viewer.show(plot)
#
# def on_click_make_it_better(obj):
#     global index, visualizations, tries
#     logger.info('Clicked remeasure, question: {}'.format(index))
#
#     key_val = [x for x in visualizations[index].items()]
#     logger.info(key_val)
#     measure_dict = {'left': key_val[0], 'right': key_val[1]}
#     binning(measure_dict, epsilon = epsilon_increments[index])
#
# submit_btn.on_click(on_click_submit)
# make_it_better.on_click(on_click_make_it_better)

In [55]:
# text_area = []
# for i in range(len(questions)):
#     text_area.append(answer)


In [56]:
back_end = src.backend.initialize_backend(cps_domain, data_path, budget=budget)
# global back_end
rm_spec = [0, 0, 0, 0, 0, 0]
rm_hit = [False, False, False, False, False, False]

def binning(measure_dict, id=None, epsilon=None):
    rm_hit[id] = True
    global tries, curr_spec, back_end, current_spec_arr, hist_workloads, plot_output_rm_arr
    
    left_col = measure_dict['left'] 
    right_col = measure_dict['right']
    
    widths = {left_col[0]:left_col[1], right_col[0]:right_col[1]}
    
    hist = builder.histogram_workload(cps_domain.config, bin_widths=widths)

    if epsilon is not None:
        if tries+1 >= max_tries:
            for i in range(len(questions)-1):
                remeasure_arr[i].close()
                budget_spent.value = "<b>You have used all the remeasures</b>"
                # progress_bar.bar_style='danger'
        elif tries+1 > max_tries:
            return
        # else:
        #     clear_output()
        y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon, restarts=20)

    prev_spec = current_spec_remeasure[id]

    # print("Spec Previous", prev_spec)
    # print("---------------------------------------------------------------")
    prev_spec.rename(columns={'error': 'error_prev', 'plus_error': 'plus_error_prev', 'minus_error': 'minus_error_prev', 'true_count':'true_count_prev', 'noisy_count':'noisy_count_prev'}, inplace=True)

    current_spec_remeasure[id], cached = back_end.display(hist) ################# Cache_search function called

    # print("spec after ", prev_spec)

    spec = current_spec_remeasure[id].join(prev_spec, on=[left_col[0], right_col[0]]).reset_index([left_col[0], right_col[0]])

    rm_spec[id] = spec
    clear_output()
    display(gap_button)
    for i in range(len(questions) - 1):
        if tries+1 < max_tries:
            bar_label = widgets.HBox([budget_spent, progress_bar, align_button_remeasure, remeasure_arr[i]])
        else:
            bar_label = widgets.HBox([budget_spent, progress_bar])
        ques = widgets.HBox([prompt_arr[i+1]])
        # prompt_answer = widgets.VBox([answer_arr[i+1], submit_btn], layout=box_layout)
        prompt_answer = widgets.VBox([answer_arr[i+1]], layout=box_layout)
        display(separator)
        display(ques)
        display(bar_label)
        if rm_hit[i]:
            key_value_new = [x for x in visualizations[i+1].items()]
            measure_dict_new = {'left': key_value_new[0], 'right': key_value_new[1]}
            left_col_new = measure_dict_new['left']
            right_col_new = measure_dict_new['right']
            plot_new = src.plots.linked_hist_test(left_col_new[0], right_col_new[0], data=rm_spec[i], projection=True,label=False)
            display(plot_new)
        else:
            column_names_new = list(visualizations[i+1].keys())
            plot_new = src.plots.linked_hist(column_names_new[0], column_names_new[1], data=current_spec_arr[i].reset_index(column_names_new), display_true=False, history=False)
            display(plot_new)
        interface = widgets.VBox([prompt_answer])
        display(interface)
    display(separator)
    display(widgets.HBox([align_button, submit_btn_final]))

    # with plot_output_arr[id]:
    #     plot_output_arr[id].clear_output()
    #     # clear_output()
    #     # plot_output_arr[id].close()
    #     plot = src.plots.linked_hist_test(left_col[0], right_col[0], data=spec, projection=True,label=False)
    #     display(plot)

    # display(plot_output_arr[id])
        # print("remeasure plot output: ", plot_output_arr[id])

    # hist_plot_rm = widgets.VBox([plot_output_arr[id]])
    # display(hist_plot_rm)

    # for i in range(len(questions) - 2):
    #     key_val2 = [x for x in visualizations[i + 2].items()]
    #     measure_dict2 = {'left': key_val2[0], 'right': key_val2[1]}
    #     left_col2 = measure_dict2['left']
    #     right_col2 = measure_dict2['right']
    #     widths = {left_col2[0]:left_col2[1], right_col2[0]:right_col2[1]}
    #
    #     prev_spec2 = current_spec_arr[i + 1]
    #     prev_spec2.rename(columns={'error': 'error_prev', 'plus_error': 'plus_error_prev', 'minus_error': 'minus_error_prev', 'true_count':'true_count_prev', 'noisy_count':'noisy_count_prev'}, inplace=True)
    #     # hist2 = hist_workloads[i + 2]
    #     hist2 = builder.histogram_workload(cps_domain.config, bin_widths=widths)
    #
    #     current_spec_arr[i + 1], cached2 = back_end.display(hist2) ################# Cache_search function called
    #
    #     spec2 = current_spec_arr[i + 1].join(prev_spec2, on=[left_col2[0], right_col2[0]]).reset_index([left_col2[0], right_col2[0]])
    #     spec2 = spec2.round(0)
    #
    #     with plot_output:
    #         # plot_output.clear_output()
    #         plot2 = src.plots.linked_hist_test(left_col2[0], right_col2[0], data=spec2, projection=True,label=False)
    #         # plot = src.plots.linked_hist(left_col[0], right_col[0], data=spec, display_true=False, history=False)
    #         display(plot2)
    
box_layout = widgets.Layout(display='flex',
    flex_flow='column',
    align_items='flex-start',
    color='black',
    width='80%')

def on_click_submit_all(obj):
    ans_dict = {}
    for i in range(len(questions) - 1):
        if answer_arr[i + 1].value == '':
            align_button.description = 'Please answer all the questions'
            align_button.style.text_color = '#E30000'
            return
        a = i+1
        ans_dict['Question ' + str(a) + ' answer'] = answer_arr[i+1].value
    with open("answers/answer.txt", 'a') as f:
        f.write('\n')
        for key, value in ans_dict.items():
            f.write('%s : %s\n' % (key, value))
        f.write('-------------------------------------------------------------')
    clear_output()
    prompt.value="<b>Survey finished, thanks for your participation!</b>"
    display(prompt)


# bar_label = widgets.HBox([budget_spent, progress_bar, make_it_better])
prompt_answer = widgets.VBox([answer, submit_btn], layout=box_layout)
submit_btn_final.on_click(on_click_submit_all)

Seed  0
Inside BackEnd  RandomState(MT19937)


In [58]:
# display(bar_label)
interface = widgets.VBox([gap_button, prompt, prompt_answer])
display(interface)

Button(disabled=True, layout=Layout(height='50px', width='95%'), style=ButtonStyle(button_color='white'))

Seed  0
Inside BackEnd  RandomState(MT19937)
RandomState(MT19937)


Button(layout=Layout(height='5px', width='99%'), style=ButtonStyle(button_color='black'))

Button(disabled=True, layout=Layout(height='1px', width='95%'), style=ButtonStyle(button_color='white'))

alt.HConcatChart(...)

...

RandomState(MT19937)


Button(layout=Layout(height='5px', width='99%'), style=ButtonStyle(button_color='black'))

Button(disabled=True, layout=Layout(height='1px', width='95%'), style=ButtonStyle(button_color='white'))

alt.HConcatChart(...)

Button(layout=Layout(height='5px', width='99%'), style=ButtonStyle(button_color='black'))